In [7]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing the datasets
training_data  = pd.read_csv('data/Train_data.csv')
test_data = pd.read_csv('data/Test_data.csv')

In [14]:
# checking the shape of the datasets
print(training_data.shape)
print(test_data.shape)

(25192, 42)
(22544, 41)


In [9]:
# checking the head of the datasets
print(training_data.head())
print(test_data.head())

   duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0         0           tcp  ftp_data   SF        491          0     0   
1         0           udp     other   SF        146          0     0   
2         0           tcp   private   S0          0          0     0   
3         0           tcp      http   SF        232       8153     0   
4         0           tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                  25   
1               0       0    0  ...                   1   
2               0       0    0  ...                  26   
3               0       0    0  ...                 255   
4               0       0    0  ...                 255   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.17                    0.03   
1                    0.00                    0.60   
2                    0.10                    0.05   


In [10]:
# checking the info of the datasets
print(training_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

In [11]:
# checking the null values of the datasets
print(training_data.isnull().sum())
print(test_data.isnull().sum())

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [12]:
# checking the unique values of the datasets
print(training_data.nunique())
print(test_data.nunique())

duration                        758
protocol_type                     3
service                          66
flag                             11
src_bytes                      1665
dst_bytes                      3922
land                              2
wrong_fragment                    3
urgent                            2
hot                              22
num_failed_logins                 5
logged_in                         2
num_compromised                  28
root_shell                        2
su_attempted                      3
num_root                         28
num_file_creations               20
num_shells                        2
num_access_files                  7
num_outbound_cmds                 1
is_host_login                     1
is_guest_login                    2
count                           466
srv_count                       414
serror_rate                      70
srv_serror_rate                  56
rerror_rate                      72
srv_rerror_rate             

In [13]:
# checking the value counts of the datasets
print(training_data['Label'].value_counts())
print(test_data['Label'].value_counts())

KeyError: 'Label'

In [ ]:
# checking the correlation of the datasets
print(training_data.corr())
print(test_data.corr())

In [ ]:
# checking the skewness of the datasets
print(training_data.skew())
print(test_data.skew())

In [ ]:
# checking the kurtosis of the datasets
print(training_data.kurt())
print(test_data.kurt())

In [ ]:
# checking the distribution of the datasets
plt.figure(figsize=(20,20))
plt.subplot(2,2,1)
sns.distplot(training_data['Duration'])
plt.subplot(2,2,2)
sns.distplot(training_data['Src_bytes'])
plt.subplot(2,2,3)
sns.distplot(training_data['Dst_bytes'])
plt.subplot(2,2,4)
sns.distplot(training_data['Land'])
plt.show()